In [1]:
#Dependencies
import os
import urllib
import json
import csv
import simplejson
import pandas as pd
import pytest
import base64
import requests
from pprint import pprint
from datetime import datetime
from ohmysportsfeedspy.v1_0 import API_v1_0
from ohmysportsfeedspy.v1_1 import API_v1_1
from ohmysportsfeedspy.v1_2 import API_v1_2
from ohmysportsfeedspy import MySportsFeeds
msf = MySportsFeeds(version="1.2")

In [16]:
NFL_data = "box_scores.csv"
NFL_data = pd.read_csv(NFL_data)
NFL_data.head(1)

,date,visitor,home,visitor_score,home_score,visitor_first_downs,visitor_rushing_first_downs,visitor_passing_first_downs,visitor_penalties,visitor_net_yards,...,home_kick_return_splits,home_int_return_splits,home_penalty_splits,home_fumble_splits,home_field_goals,home_third_down_splits,home_fourth_down_splits,home_total_plays,home_avg_gain,home_time_of_possession
0,"September 7, 2014",Cleveland,Pittsburgh,27,30,23,9,11,3,389,...,2-29,0-0,11-96,1-0,3-3,4-12-33%,1-1-100%,67,7.5,32:27


In [54]:
team_scores = NFL_data[["date","visitor","home","visitor_score","home_score"]]
team_scores.head(1)

,date,visitor,home,visitor_score,home_score
0,"September 7, 2014",Cleveland,Pittsburgh,27,30


In [81]:
NFL_games = "nfl_games.csv"
NFL_games = pd.read_csv(NFL_games)
NFL_date_games = NFL_games.set_index("date")

NFL_date_games = NFL_date_games["2014-09-04":]

In [98]:
NFL_date_games = NFL_date_games[["season","team1","team2","score1","score2"]]
nfl_scores = NFL_date_games
nfl_scores.head()

,season,team1,team2,score1,score2
date,,,,,
2014-09-04,2014,SEA,GB,36,16
2014-09-07,2014,PHI,JAX,34,17
2014-09-07,2014,PIT,CLE,30,27
2014-09-07,2014,BAL,CIN,16,23
2014-09-07,2014,TB,CAR,14,20


In [122]:
team1_scores = nfl_score.groupby("team1").sum()["score1"]
team2_scores = nfl_score.groupby("team2").sum()["score2"]

team1_scores_df = pd.DataFrame(team1_scores)
team2_scores_df = pd.DataFrame(team2_scores)

In [123]:
team2_scores_df.head(1)

,score2
team2,
ARI,655


In [124]:
team1_scores_df.head(1)

,score1
team1,
ARI,619


In [155]:
#Scores from 2014 to 2017
nfl_team_scores = pd.concat([team1_scores_df, team2_scores_df], axis=1)
nfl_team_scores["Total Points from 2014 to 2017"] = nfl_team_scores["score1"] + nfl_team_scores["score2"]
del nfl_team_scores["score1"]
del nfl_team_scores["score2"]
nfl_team_scores

,Total Points from 2014 to 2017
ARI,1274
ATL,1368
BAL,1141
BUF,1121
CAR,1342
CHI,933
CIN,1135
CLE,841
DAL,1239
DEN,1250
